In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# Define dataset transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

# Download and load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

classes = trainset.classes


Files already downloaded and verified
Files already downloaded and verified


In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
model = CNN()


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

print("Finished Training")

[1, 100] loss: 1.907
[1, 200] loss: 1.554
[1, 300] loss: 1.482
[1, 400] loss: 1.390
[1, 500] loss: 1.318
[1, 600] loss: 1.291
[1, 700] loss: 1.221
[1, 800] loss: 1.214
[1, 900] loss: 1.141
[1, 1000] loss: 1.107
[1, 1100] loss: 1.111
[1, 1200] loss: 1.081
[1, 1300] loss: 1.087
[1, 1400] loss: 1.039
[1, 1500] loss: 0.998
[2, 100] loss: 0.953
[2, 200] loss: 0.936
[2, 300] loss: 0.955
[2, 400] loss: 0.902
[2, 500] loss: 0.908
[2, 600] loss: 0.919
[2, 700] loss: 0.882
[2, 800] loss: 0.886
[2, 900] loss: 0.890
[2, 1000] loss: 0.851
[2, 1100] loss: 0.874
[2, 1200] loss: 0.818
[2, 1300] loss: 0.844
[2, 1400] loss: 0.857
[2, 1500] loss: 0.875
[3, 100] loss: 0.726
[3, 200] loss: 0.737
[3, 300] loss: 0.744
[3, 400] loss: 0.723
[3, 500] loss: 0.730
[3, 600] loss: 0.721
[3, 700] loss: 0.738
[3, 800] loss: 0.768
[3, 900] loss: 0.701
[3, 1000] loss: 0.729
[3, 1100] loss: 0.754
[3, 1200] loss: 0.743
[3, 1300] loss: 0.747
[3, 1400] loss: 0.714
[3, 1500] loss: 0.732
[4, 100] loss: 0.599
[4, 200] loss: 0

In [9]:
# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")


Accuracy on test set: 72.80%


In [10]:
# Fine-tuning a pretrained model
from torchvision import models

pretrained_model = models.resnet18(pretrained=True)
pretrained_model.fc = nn.Linear(pretrained_model.fc.in_features, 10)

# Freeze layers except the final fully connected layer
for param in pretrained_model.parameters():
    param.requires_grad = False

for param in pretrained_model.fc.parameters():
    param.requires_grad = True

# Define optimizer and criterion for pretrained model
optimizer = optim.Adam(pretrained_model.fc.parameters(), lr=0.001)

# Similar training loop can be implemented as before for fine-tuning



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 180MB/s]
